Ucitavamo dataset

In [1]:
import torch
import numpy as np 
from torch.utils.data import TensorDataset

ulaz= np.loadtxt('TrainDataset.txt')
podaci=ulaz[:,1:]
#print(podaci)
b=np.max(podaci,axis=0)
podaci=podaci/b
#print(podaci)
#print(np.max(podaci),np.min(podaci))
obelezja=ulaz[:,:1]
#print(obelezja)
podaci=torch.Tensor(podaci)
obelezja=torch.Tensor(obelezja)
obelezja=obelezja.flatten()

dataset=TensorDataset(podaci,obelezja)
train_set,test_set = torch.utils.data.random_split(dataset,[0.9,0.1])

print(len(test_set))
print(test_set[0])
print(test_set[0][0][2])

12500
(tensor([0.5717, 0.5584, 0.5254,  ..., 0.0000, 0.0204, 0.0899]), tensor(40.))
tensor(0.5254)


In [ ]:
obelezja=obelezja.flatten()
obelezja=obelezja.()
print(obelezja)

In [2]:
if torch.cuda.is_available():
    device="cuda"
else:
    device="cpu"
    print("NE RADI CUDA!")

Inicijalizacija mreze

In [3]:
import torch.nn as nn

class Mreza (nn.Module):
    def __init__ (self):
        super().__init__()
        self.act=nn.Sequential(
            nn.Linear(1564,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000,1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),

            nn.Linear(1000,300),
            nn.BatchNorm1d(300),
            nn.ReLU(),
            nn.Linear(300,120),
            nn.BatchNorm1d(120),
            nn.ReLU(),
            nn.Linear(120,50),
            nn.BatchNorm1d(50),
            nn.ReLU()
        )
        self.act=self.act.to(device)
    def forward (self, x):
        return self.act(x)
model = Mreza()
model = model.to(device)

Testiranje

In [4]:
def totensor(y):
    ret=np.zeros((len(y),50))
   # print(y[0])
   # print(y[0][0])
   # print(y[0][0].item())
    for i in range(0,len(y)):
        ret[i][round(y[i].item())]=1
    return torch.Tensor(ret).to(device)

def test(model,sample):
    #print(sample)
    sample=sample.to(device)
    rez=model(sample.reshape(1,1564))
    _,b=torch.max(rez,dim=1)
    return b


Inicijalizacija parametara

In [5]:
broj_epoha = 50
batch_velicina=500
loss_funk = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_velicina,shuffle=True,num_workers=0,drop_last=True,pin_memory=True)
val_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_velicina,shuffle=False,num_workers=0,drop_last=False,pin_memory=True)


In [6]:
def get_accuracy(y_hat,y):
   # print(y)
    y_hat = y_hat.cpu().detach().numpy()
    y=y.cpu().numpy()
    y_hat=np.argmax(y_hat,axis=1)
   # print(y_hat,y)
    corrects=(y_hat==y).sum()
   # print(corrects)
    return corrects.item()

def validation(model):
    with torch.no_grad():
        model.eval()
        kol=0
        total_loss=0
        for batch_num, (x, y) in enumerate(val_loader):
                x = x.to(device)
                y = y.to(device)
                #print(y)
                y_hat = model(x)
                loss=loss_funk(y_hat,totensor(y))
                total_loss+=loss.detach().item()
                kol+=get_accuracy(y_hat=y_hat, y=y)/len(y)
                del y_hat
        kol=kol*100/batch_num
        total_loss/=batch_num
    return total_loss,kol

Treniranje

In [7]:
# MAKE SURE TO INITIALIZE PARAMETERS BEFORE STARTING TRAINING
import time
for epoha in range(1,broj_epoha+1):
    train_running_loss =0.0
    train_acc=0.0
    model = model.train() 
    start=time.time()
    for batch_num, (x, y) in enumerate(loader):
            x = x.to(device)
            y = y.to(device)
           # print(y)
            #print(y)
            y_hat = model(x)
            loss = loss_funk(y_hat, totensor(y))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            train_running_loss += loss.detach().item()
            train_acc += get_accuracy(y_hat=y_hat, y=y)/len(y)
            del y_hat

            print(f'Epoch: {epoha}\t{100 * (batch_num + 1) / len(loader):.2f}% complete. {time.time() - start:.2f} seconds elapsed in epoch. Time until completion {(time.time()-start)/((batch_num + 1) / len(loader)) - time.time()+start:.2f} seconds.',
                end='\r')

    print(f'',end='\n')    
    epoch_loss = train_running_loss / batch_num
    epoch_acc = 100*train_acc / batch_num
    val_loss,val_acc = validation(model)
    print('Epoch: %d | Train Loss: %.4f | Train Accuracy: %.2f | Validation Loss: %.4f | Validation Accuracy: %.2f' \
          %(epoha, epoch_loss, epoch_acc,val_loss,val_acc))

Epoch: 1	100.00% complete. 9.57 seconds elapsed in epoch. Time until completion 0.00 seconds.
Epoch: 1 | Train Loss: 3.6958 | Train Accuracy: 7.88 | Validation Loss: 3.6934 | Validation Accuracy: 10.08
Epoch: 2	100.00% complete. 7.40 seconds elapsed in epoch. Time until completion 0.00 seconds.
Epoch: 2 | Train Loss: 3.4012 | Train Accuracy: 12.98 | Validation Loss: 3.5231 | Validation Accuracy: 13.10
Epoch: 3	100.00% complete. 7.35 seconds elapsed in epoch. Time until completion 0.00 seconds.
Epoch: 3 | Train Loss: 3.2659 | Train Accuracy: 15.66 | Validation Loss: 3.4028 | Validation Accuracy: 16.11
Epoch: 4	100.00% complete. 7.18 seconds elapsed in epoch. Time until completion 0.00 seconds.
Epoch: 4 | Train Loss: 3.1611 | Train Accuracy: 17.74 | Validation Loss: 3.3746 | Validation Accuracy: 16.44
Epoch: 5	100.00% complete. 7.18 seconds elapsed in epoch. Time until completion 0.00 seconds.
Epoch: 5 | Train Loss: 3.0741 | Train Accuracy: 19.67 | Validation Loss: 3.3126 | Validation Ac

In [ ]:
def testt(model):
        kol=0
        for i in range(0,len(test_set)):
            out=test(model,test_set[i][0].reshape(1,1564))
            ans=test_set[i][1].item()
            if round(ans)==out:
                kol+=1
        return kol
print(testt(model))
print(testt(model)/len(test_set)*100)

In [ ]:
print(validation(model))